In [13]:
import pandas as pd
import json
import joblib

file = open('NBA Team Data.json')
teams = json.load(file)
current_nba_data = pd.read_csv('../Data/Current NBA Game Data.csv', index_col=0)

## Getting Prediction Stats
Run Current Data Scraper.ipynb get the most up to date stats for all teams. 

The program will then process the data, feed it to the machine learning model, and make a prediction!

In [14]:
# retrieves and formats recent NBA game data into a pandas data frame
def get_prediction_stats(away_team, home_team):
    away_team_data = current_nba_data.loc[teams[away_team]]
    away_team_data.index = [f'{i}_A' for i in list(away_team_data.index)]
    home_team_data = current_nba_data.loc[teams[home_team]]
    home_team_data.index = [f'{i}_H' for i in list(home_team_data.index)]

    game_prediction_data = pd.concat([away_team_data, home_team_data])
    game_prediction_data['NEXT_HOME/AWAY'] = 0.0
    game_prediction_data['AWAY_TEAM'] = away_team
    game_prediction_data['HOME_TEAM'] = home_team


    return pd.DataFrame(game_prediction_data).T

In [15]:
def make_prediction(away_team, home_team):    
    game_prediction_data = get_prediction_stats(away_team, home_team)

    # loads the saved preprocessing pipeline and ml model
    scaler, rr, training_values, target = joblib.load('Ridge Classifier Model.pkl')

    # applies the preprocessing pipeline to current game prediction data and make a prediciton
    scaled_data = pd.DataFrame(scaler.transform(game_prediction_data[training_values]), columns=training_values)
    game_prediction_data["PREDICTED_WINNER"] = rr.predict(scaled_data)

    # finds the predicted winning team
    win_team = away_team if game_prediction_data["PREDICTED_WINNER"][0] == 1.0 else home_team
    return win_team

## Making a Prediction!
Run make_prediction with team abbreviations. 
The rr model will then print it's prediction for that given game.

NOTE : Make sure you specify which teams is home and which is away to increase the accuracy of the prediction 

In [16]:
make_prediction(away_team='DET', home_team='MIN')

'MIN'

In [18]:
def make_multiple_predictions():
    running = True
    while running:
        # selecting away team
        a_team , h_team = '', ''
        while(a_team not in teams):
            a_team = input('AWAY TEAM (q to quit): ')
            if a_team == 'q':
                running = False
                break
            a_team = a_team.upper()

        if not running : break 
        print(f'AWAY TEAM : {a_team}')

        while(h_team not in teams):
            h_team = input('HOME TEAM (q to quit): ')
            if h_team == 'q':
                running = False
                break
            h_team = h_team.upper()

        if not running : break 
        print(f'HOME TEAM : {h_team}')

        print(f'{a_team} at {h_team}       Predicted Winner : {make_prediction(away_team=a_team, home_team=h_team)}')

    print('THANKS FOR USING NBA GAME PREDICTOR! 🏀')

make_multiple_predictions()

AWAY TEAM : DET
HOME TEAM : DAL
DET at DAL       Predicted Winner : DAL
AWAY TEAM : LAL
HOME TEAM : LAC
LAL at LAC       Predicted Winner : LAC
AWAY TEAM : NYK
HOME TEAM : BKN
NYK at BKN       Predicted Winner : BKN
AWAY TEAM : SAS
HOME TEAM : HOU
SAS at HOU       Predicted Winner : HOU
THANKS FOR USING NBA GAME PREDICTOR! 🏀
